In [1]:
import re
import numpy
import pandas
import string

from bs4 import BeautifulSoup

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

import warnings; warnings.simplefilter('ignore')

In [20]:
def expandContractions(text):
    contractionList = {
      "ain't": "am not",
      "aren't": "are not",
      "can't": "cannot",
      "'cause": "because",
      "could've": "could have",
      "couldn't": "could not",
      "didn't": "did not",
      "doesn't": "does not",
      "don't": "do not",
      "hadn't": "had not",
      "hasn't": "has not",
      "haven't": "have not",
      "he'd": "he would",
      "he'll": "he will",
      "he's": "he is",
      "how'll": "how will",
      "how's": "how is",
      "I'd": "I would",
      "I'll": "I will",
      "I'm": "I am",
      "I've": "I have",
      "isn't": "is not",
      "it'll": "it will",
      "it's": "it is",
      "let's": "let us",
      "might've": "might have",
      "must've": "must have",
      "mustn't": "must not",
      "needn't": "need not",
      "never'll":"never will",
      "she'd": "she would",
      "she'll": "she will",
      "she's": "she is",
      "should've": "should have",
      "shouldn't": "should not",
      "that'd": "that would",
      "that's": "that is",
      "there's": "there is",
      "they'd": "they would",
      "they'll": "they will",
      "they're": "they are",
      "they've": "they have",
      "wasn't": "was not",
      "we'll": "we will",
      "we're": "we are",
      "we've": "we have",
      "weren't": "were not",
      "what'll": "what will",
      "what're": "what are",
      "what's": "what is",
      "what've": "what have",
      "when's": "when is",
      "where'd": "where did",
      "where's": "where is",
      "who'll": "who will",
      "who's": "who is",
      "will've": "will have",
      "won't": "will not",
      "would've": "would have",
      "wouldn't": "would not",
      "you'd": "you had",
      "you'll": "you will",
      "you're": "you are",
      "you've": "you have"
    }
    contraction = re.compile('(%s)' % '|'.join(contractionList.keys()))
    def replace(match):
        return contractionList[match.group(0)]
    return contraction.sub(replace, text)

In [21]:
def setstopwords():
    stop_words = stopwords.words('english')
    stop_words.append('URL')
    stop_words.remove('no')
    stop_words.remove('nor')
    stop_words.remove('not')
    return stop_words

In [22]:
def preprocessing(text):
    stop_words = setstopwords()
    #porter = PorterStemmer("english")
    try:
        text = re.sub('[0-9]+', ' ', text)
        text = BeautifulSoup(text, "lxml")
        text = text.get_text()
        text = text.lower()
        text = expandContractions(text)
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','URL',text)
        text = re.sub('[\s]+', ' ', text)
        text = re.sub('['+string.punctuation+']', ' ', text)
        text = text.strip('\'"')
        text = " ".join(text.split())
        text = " ".join([word for word in text.split() if word not in stop_words])
        return text
    except:
        return []

In [24]:
def splitData(data):
    _reviewSummary = []
    _reviewText = []
    _reviewSentiment = []
    for index,row in data.iterrows():
        _sentiment = row['Sentiment']
        _summary = preprocessing(row['Summary'])
        _text = preprocessing(row['Text'])
        #print(_sentiment)
        #print(_summary)
        #print(_text)
        if len(_summary) > 0 and len(_text) > 0:
            _reviewSummary.append(_summary)
            _reviewText.append(_text)
            _reviewSentiment.append(_sentiment)
    return [_reviewSummary,_reviewText,_reviewSentiment]

In [6]:
def build_classifier(classifier_name):
    classifier_pipeline = []
    classifier_pipeline.append(('vectorizer', CountVectorizer()))
    classifier_pipeline.append(('transformer', TfidfTransformer(use_idf = True)))

    if classifier_name == 'LinearSVC':
        classifier_pipeline.append(('clsf', LinearSVC()))
    elif classifier_name == 'MultinomialNB':
        classifier_pipeline.append(('clsf', MultinomialNB()))
    elif classifier_name == 'LogisticRegression':
        classifier_pipeline.append(('clsf', LogisticRegression()))
    elif classifier_name == 'RandomForest':
        classifier_pipeline.append(('clsf', RandomForestClassifier(n_estimators=10)))
    elif classifier_name == 'AdaBoost':
        classifier_pipeline.append(('clsf', AdaBoostClassifier(n_estimators=100)))
    elif classifier_name == 'SVMRadialKernel':
        classifier_pipeline.append(('clsf', SVC(kernel='rbf')))
    elif classifier_name == 'SVMPolyKernel':
        classifier_pipeline.append(('clsf', SVC(kernel='poly', degree=3)))
    
    return Pipeline(classifier_pipeline)

In [7]:
def classify(data, sentiment, classifiers, k = 10):

    kfold_res = {}

    for classifier_name in classifiers:
        kf = KFold(n_splits = k)
        precision = 0.0
        recall = 0.0
        f1score = 0.0
        accuracy = 0.0

        for train_id, test_id in kf.split(data):
            train_data = [data[i] for i in train_id]
            train_class = [sentiment[i] for i in train_id]
            test_data = [data[i] for i in test_id]
            test_class = [sentiment[i] for i in test_id]
            
            classifier = build_classifier(classifier_name)
            classifier.fit(train_data, train_class)
            
            prediction = classifier.predict(test_data)
            
            precision += precision_score(test_class, prediction, average="macro")
            recall += recall_score(test_class, prediction, average="macro")
            f1score += f1_score(test_class, prediction, average="macro")
            accuracy += accuracy_score(test_class, prediction)

        kfold_res[classifier_name] = {'precision': (precision/k), 'recall': (recall/k), 'f1score':(f1score/k), 'accuracy':(accuracy/k)}

    return kfold_res

In [25]:
path = '../Data/Reviews.csv'
reviews = pandas.read_csv(path, usecols=['Summary','Text','Score'])
reviews["Sentiment"] = reviews["Score"].apply(lambda score: "positive" if score > 3 else "negative")
print('Review data frame shape : ', reviews.shape)
print(reviews.head(5))
print('=============================================================================================')
print()

reviewsSubset = reviews.head(1000)
print(reviewsSubset.shape)
[summary,text,sentiment] = splitData(reviewsSubset)

classifiers = ['LogisticRegression', 'MultinomialNB', 'LinearSVC', 'AdaBoost', 'RandomForest', 'SVMRadialKernel']

_TextSentiment = classify(text, sentiment, classifiers, k=10)
_SummarySentiment = classify(summary, sentiment, classifiers, k=10)

for classifier in classifiers:
    print(classifier)
    print('\t\tPrecision - Summary = ', round(_SummarySentiment[classifier]['precision'], 2), '\t\tPrecision - Text = ', round(_TextSentiment[classifier]['precision'],2))
    print('\t\tRecall - Summary = ', round(_SummarySentiment[classifier]['recall'],2), '\t\tRecall - Text = ', round(_TextSentiment[classifier]['recall'],2))
    print('\t\tF1_score - Summary = ', round(_SummarySentiment[classifier]['f1score'],2), '\t\tF1_score - Text = ', round(_TextSentiment[classifier]['f1score'],2))
    print('\t\tAccuracy - Summary = ', round(_SummarySentiment[classifier]['accuracy'],2), '\t\tAccuracy - Text = ', round(_TextSentiment[classifier]['accuracy'],2))
    print('---------------------------------------------------------------------------------------------')

Review data frame shape :  (568454, 4)
   Score                Summary  \
0      5  Good Quality Dog Food   
1      1      Not as Advertised   
2      4  "Delight" says it all   
3      2         Cough Medicine   
4      5            Great taffy   

                                                Text Sentiment  
0  I have bought several of the Vitality canned d...  positive  
1  Product arrived labeled as Jumbo Salted Peanut...  negative  
2  This is a confection that has been around a fe...  positive  
3  If you are looking for the secret ingredient i...  negative  
4  Great taffy at a great price.  There was a wid...  positive  

(1000, 4)
['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better', 'product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted not sure error vendor intended represent product jumbo', 'confection around centuries light pill